In [1]:
import sqlite_utils
import llm

model = llm.get_model("gpt-4.1-mini")

def text_to_sql(db: sqlite_utils.Database, question: str) -> str:
    """Convert a prompt to SQL using the LLM."""
    prompt = "Schema:\n\n{}\n\nQuestion:\n\n{}".format(
        db.schema, question
    )
    return model.prompt(
        prompt,
        system="reply with SQLite SQL, not in markdown, just the SQL",
    ).text()

db = sqlite_utils.Database(llm.user_dir() / "logs.db")

sql = text_to_sql(db, "how many conversations are there?")

print(sql)

# Now execute it
result = db.query(sql)
print(list(result))

SELECT COUNT(*) FROM conversations;
[{'COUNT(*)': 6}]


# Upgrading that to a CLI tool

In [2]:
import argparse
from pathlib import Path

import sqlite_utils
import llm

# pick your model
model = llm.get_model("gpt-4.1-mini")

def text_to_sql(db: sqlite_utils.Database, question: str) -> str:
    """Convert an English question into a SQLite SQL statement."""
    prompt = "Schema:\n\n{}\n\nQuestion:\n\n{}".format(db.schema, question)
    resp = model.prompt(
        prompt,
        system="reply with SQLite SQL, not in markdown, just the SQL",
    )
    return resp.text().strip()

def main():
    parser = argparse.ArgumentParser(
        description="Turn a natural-language question into SQL (and optionally run it)."
    )
    parser.add_argument(
        "question",
        help="The question to ask of your SQLite database, in plain English.",
    )
    parser.add_argument(
        "--db",
        "-d",
        default=str(llm.user_dir() / "logs.db"),
        help="Path to the SQLite database file.  [default: %(default)s]",
    )
    parser.add_argument(
        "--execute",
        "-x",
        action="store_true",
        help="Execute the generated SQL and print the results instead of just showing the SQL.",
    )
    args = parser.parse_args()

    db_path = Path(args.db)
    if not db_path.exists():
        parser.error(f"Database file not found: {db_path!r}")

    db = sqlite_utils.Database(db_path)
    sql = text_to_sql(db, args.question)

    if args.execute:
        try:
            rows = list(db.query(sql))
        except Exception as e:
            print("ERROR running SQL:", e)
            print("SQL was:", sql)
            raise SystemExit(1)
        # print rows as simple CSV
        for row in rows:
            print(row)
    else:
        print(sql)

if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] [--db DB] [--execute] question
ipykernel_launcher.py: error: the following arguments are required: question


SystemExit: 2

/workspaces/python-3.13-playground/venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3680: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
import llm, json
from pydantic import BaseModel

class Pelican(BaseModel):
    name: str
    age: int
    short_bio: str
    beak_capacity_ml: float

model = llm.get_model("gpt-4o-mini")
response = model.prompt("Describe a spectacular pelican", schema=Pelican)
pelican = json.loads(response.text())
print(pelican)

{'name': 'Spectacular Pelican', 'age': 5, 'short_bio': 'This pelican is known for its majestic wingspan that exceeds 8 feet, making it one of the largest birds in the world. It is often seen gliding gracefully over coastal waters and performing impressive dives to catch fish.', 'beak_capacity_ml': 800}
